In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("D:\SQL\TRIALSF3.csv")

In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1.0,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1.0,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1.0,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1.0,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1.0,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


In [4]:
table1=df[['amount','oldbalanceOrg', 'newbalanceOrig','oldbalanceDest', 'newbalanceDest', 'isFraud', 'type']]

In [5]:
table1.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type
0,9839.64,170136.0,160296.36,0.0,0.0,0.0,PAYMENT
1,1864.28,21249.0,19384.72,0.0,0.0,0.0,PAYMENT
2,181.00,181.0,0.00,0.0,0.0,1.0,TRANSFER
3,181.00,181.0,0.00,21182.0,0.0,1.0,CASH_OUT
4,11668.14,41554.0,29885.86,0.0,0.0,0.0,PAYMENT


In [6]:
X=table1.iloc[:,1:]
y=table1.iloc[:,0]

In [7]:
X.head()

,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type
0,170136.0,160296.36,0.0,0.0,0.0,PAYMENT
1,21249.0,19384.72,0.0,0.0,0.0,PAYMENT
2,181.0,0.00,0.0,0.0,1.0,TRANSFER
3,181.0,0.00,21182.0,0.0,1.0,CASH_OUT
4,41554.0,29885.86,0.0,0.0,0.0,PAYMENT


In [8]:
y.head()

0     9839.64
1     1864.28
2      181.00
3      181.00
4    11668.14
Name: amount, dtype: float64

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [10]:
from sklearn.ensemble import RandomForestRegressor
rf_random=RandomForestRegressor()

In [11]:
import numpy as np
#hyper parameters tuning
n_estimators=[int(x) for x in np.linspace(start=100, stop=1200, num=12)]
print(n_estimators)

[100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200]


In [12]:
from sklearn.model_selection import RandomizedSearchCV

In [13]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10]

In [14]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200], 'max_features': ['auto', 'sqrt'], 'max_depth': [5, 10, 15, 20, 25, 30], 'min_samples_split': [2, 5, 10, 15, 100], 'min_samples_leaf': [1, 2, 5, 10]}


In [15]:
rf=RandomForestRegressor()

In [16]:
rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [17]:
dummies=pd.get_dummies(table1[["type"]])
dummies

,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,0,1
3,0,1,0,0,0
4,0,0,0,1,0
...,...,...,...,...,...
1048570,0,1,0,0,0
1048571,0,0,0,1,0
1048572,0,0,0,1,0
1048573,0,0,0,1,0


In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
table1["type"] = pd.to_numeric(table1["type"], errors='coerce', downcast="float")

In [20]:
print(table1.dtypes)

amount            float64
oldbalanceOrg     float64
newbalanceOrig    float64
oldbalanceDest    float64
newbalanceDest    float64
isFraud           float64
type              float32
dtype: object


In [23]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features=["type"]
one_hot=OneHotEncoder()
transformer=ColumnTransformer([("one_hot",one_hot,categorical_features)],remainder="passthrough")
transformed_X=transformer.fit_transform(X)
transformed_X
pd.DataFrame(transformed_X)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,1.0,0.0,170136.00,160296.36,0.00,0.00,0.0
1,0.0,0.0,0.0,1.0,0.0,21249.00,19384.72,0.00,0.00,0.0
2,0.0,0.0,0.0,0.0,1.0,181.00,0.00,0.00,0.00,1.0
3,0.0,1.0,0.0,0.0,0.0,181.00,0.00,21182.00,0.00,1.0
4,0.0,0.0,0.0,1.0,0.0,41554.00,29885.86,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...
1048570,0.0,1.0,0.0,0.0,0.0,479803.00,347245.65,484329.37,616886.72,0.0
1048571,0.0,0.0,0.0,1.0,0.0,90545.00,80627.64,0.00,0.00,0.0
1048572,0.0,0.0,0.0,1.0,0.0,20545.00,6404.95,0.00,0.00,0.0
1048573,0.0,0.0,0.0,1.0,0.0,90605.00,80584.95,0.00,0.00,0.0
